In [ ]:
import pandas as pd
import os


Conectando ao Neo4j 

In [4]:
from neo4j import GraphDatabase

# Neo4j variables
NEO4J_URL = os.getenv("NEO4J_URL")
NEO4J_USERNAME =os.getenv("NEO4J_USERNAME")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD")


graph = GraphDatabase.driver(NEO4J_URL, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

Buscando Exerpts duplicados na base, aplicar todas as relacões em um deles e deletar o outro.

In [58]:
for n in range(191):

    # Query to find next exerpts with the same URI
    query1 = """
    match (n:Exerpt) 
    match (m:Exerpt) 
    where ID(n) <> ID(m) and n.uri = m.uri 
    return ID(n), ID(m)
    limit 1
    """

    graph = GraphDatabase.driver(NEO4J_URL, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))
    results = graph.execute_query(query1).records
    graph.close()

    exerpt = results[0][0]
    same_uri = results[0][1]

    # Query to merge mentions to the exerpts with the same URI 
    query2 = f"""
    match (m)-[r:mentions]-(other_node) 
    where ID(m) = {exerpt}	 
    match (n) where ID(n) = {same_uri}
    merge (n)-[:mentions]-(other_node)
    """
    graph = GraphDatabase.driver(NEO4J_URL, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))
    results = graph.execute_query(query2).records
    graph.close()

    #Query to delete the first exerpt
    query3 = f"""
    match (m) 
    where ID(m) = {exerpt}
    detach delete m
    """
    graph = GraphDatabase.driver(NEO4J_URL, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))
    results = graph.execute_query(query3).records
    graph.close()

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The query used a deprecated function: `id`.} {position: line: 4, column: 11, offset: 55} for query: '\n    match (n:Exerpt) \n    match (m:Exerpt) \n    where ID(n) <> ID(m) and n.uri = m.uri \n    return ID(n), ID(m)\n    limit 1\n    '
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The query used a deprecated function: `id`.} {position: line: 4, column: 20, offset: 64} for query: '\n    match (n:Exerpt) \n    match (m:Exerpt) \n    where ID(n) <> ID(m) and n.uri = m.uri \n    return ID(n), ID(m)\n    limit 1\n    '
Received notification from DBMS server: {s

IndexError: list index out of range